# 0 - BIBLIOTECAS

In [ ]:
from time import time

import pandas as pd
import torch
import yaml
from torch import nn
from torch.optim import Adam

from src.utils import get_path_projeto

# 1 - DADOS

In [ ]:
dir_projeto = get_path_projeto()

with open(dir_projeto / "config/csv_config.yaml", "r") as config_file:
    config_csv = yaml.safe_load(config_file)

dataset = pd.read_csv(dir_projeto / "data/staged/dados_empilhados.csv", **config_csv)
display(dataset.head())

,interval_start_local,interval_end_local,solar,wind,geothermal,biomass,biogas,small_hydro,coal,nuclear,natural_gas,large_hydro,batteries,imports,other
0,2023-11-01 00:00:00-03:00,2023-11-01 00:05:00-03:00,-53,1005,787.0,123,169,246,0,1100,15327,2842,1928,3000,0.0
1,2023-11-01 00:05:00-03:00,2023-11-01 00:10:00-03:00,-52,1015,788.0,123,169,243,0,1101,15213,2783,2015,3064,0.0
2,2023-11-01 00:10:00-03:00,2023-11-01 00:15:00-03:00,-52,1024,789.0,123,168,223,0,1101,15217,2762,1862,3193,0.0
3,2023-11-01 00:15:00-03:00,2023-11-01 00:20:00-03:00,-52,1036,789.0,123,169,224,0,1100,15267,2767,1830,3126,0.0
4,2023-11-01 00:20:00-03:00,2023-11-01 00:25:00-03:00,-52,1051,789.0,123,169,224,0,1101,15279,2772,1813,3078,0.0


In [3]:
wind_power_generation = dataset.loc[:, ["interval_start_local", "wind"]]
wind_power_generation.rename(
    columns={"interval_start_local": "date", "wind": "power_generation"},
    inplace=True
)
display(wind_power_generation.head())

,date,power_generation
0,2023-11-01 00:00:00-03:00,1005
1,2023-11-01 00:05:00-03:00,1015
2,2023-11-01 00:10:00-03:00,1024
3,2023-11-01 00:15:00-03:00,1036
4,2023-11-01 00:20:00-03:00,1051


In [4]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X, y = [], []
    for i in range(len(dataset) - lookback):
        feature = dataset[i : i + lookback]
        target = dataset[i + 1 : i + lookback + 1]
        X.append(feature)
        y.append(target)
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


X, y = create_dataset(
    dataset=wind_power_generation["power_generation"].values,
    lookback=1
)

/var/folders/g4/6w_d4rjs17n_cdn1lnl_nnf00000gn/T/ipykernel_4023/1936927444.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:281.)
  return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)


# 2 - REDE NEURAL

In [5]:
class ArquiteturaRedeNeural(nn.Module):
    def __init__(self) -> None:
        super(ArquiteturaRedeNeural, self).__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=50, num_layers=1, batch_first=True)
        self.linear = nn.Linear(50, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.linear(x)
        return x

In [6]:
rede_neural = ArquiteturaRedeNeural()
loss_fn = nn.L1Loss()
otimizador = Adam(params=rede_neural.parameters(), lr=0.01)

num_epochs = 100
t0 = time()

for epoch in range(num_epochs):
    y_pred = rede_neural(X)
    loss = loss_fn(y_pred, y)
    loss.backward()
    otimizador.step()
    otimizador.zero_grad()
    if (epoch + 1) % 10 == 0:
        print(f"Epoch ({epoch}/{num_epochs}): loss = {loss:.4f} | {time() - t0:.4f} s")
        t0 = time()

KeyboardInterrupt: 